In [2]:
import pandas as pd
import os

if os.name == 'nt':
    base_dir = "D:/Dropbox/Dropbox (Personal)/College/DR_Paper/"
else:
    base_dir = "/home/j/Dropbox/College/DR_Paper/"

## INPUTS
hs96isic3 = base_dir + 'Tariff Conversion/HS1996toISIC3/HS1996toISIC3.csv'
wtoandcaftadata = base_dir +"cafta-dr/Output/wtoandcaftahstariff.csv"
companydata = base_dir + "DirectoryofCompaniesandEstablishments/empresasbymunici.csv'
compoutcsv = base_dir + 'companiestariffs.csv'

## INTERMEDIATE DATA

## OUTPUTS



In [4]:
## Build table to convert HS1996 to ISIC3 data
isic3 = pd.read_csv(hs96isic3, 
                    converters={'HS 1996 Product Code': lambda x: str(x),
                                'ISIC Revision 3 Product Code': lambda x: str(x)})
isic3.columns=['HS96', 'HSdesc', 'ISIC3', 'ISICdesc']
isic3 = isic3[['HS96', 'ISIC3']]

isic3conv = dict(zip(isic3['HS96'],isic3['ISIC3']))

In [29]:
## Need to rewrite this

compdir = base_dir + "DirectoryofCompaniesandEstablishments/"
compoutcsv = base_dir + 'companiestariffs.csv'

## Merge together tariff data and Empresas by municipality
compdirdf = pd.read_csv(wtoandcaftadata)

lines = [a for a in compdirdf.columns if 'Lines' in a]
compdirdf = compdirdf.drop(lines, 1)

#Convert HS2002 codes into ISIC3 codes
compdirdf['ISIC'] = compdirdf['HS6'].apply(lambda x: "'"+isic3conv[x.replace("'","")])       
compdirdf.drop('HS6',1)
compdirdf = compdirdf.groupby('ISIC', as_index=False)[list(compdirdf.columns)].mean()

#Merge with municipality level company data
companiesdf = pd.read_csv(companydata)

companiesdf = companiesdf.merge(compdirdf, on="ISIC",how='left')
companiesdf.to_csv(compoutcsv, index=False)

In [ ]:
## Part below copied from area share companies on 1/27/16 - Need to fix this script anyway
## Compute weighted average of import competing tariff for each
## municipality/province
comptarcsv = '/home/j/Dropbox/College/DR_Paper/companiestariffs.csv'
municicsv = '/home/j/Dropbox/College/DR_Paper/municitariffs.csv'
provcsv = '/home/j/Dropbox/College/DR_Paper/provtariffs.csv'

compdf = pd.read_csv(comptarcsv)

# Reduces data frame down to only "import competing" (wink) industries
# You want to look into this later, of course
compdf = compdf[compdf['duty2007'].astype(str) != 'nan'] 

#Calculate "new" number of companies in province and municipality
compdf = compdf.drop(['MUNICINUM','PROVNUM'],1)
calcdf = compdf
calcdf = calcdf[list(calcdf.columns)[:4]]
municidf2 = calcdf.groupby(['PROVINCE','MUNICIPIO'], as_index=False).sum()
provdf2 = calcdf.groupby('PROVINCE', as_index=False)['EMPRESAS'].sum()
municidf2.columns = ['PROVINCE','MUNICIPIO','MUNICINUM']
provdf2.columns = ['PROVINCE','PROVNUM']

compdf = compdf.merge(municidf2, on=['PROVINCE','MUNICIPIO'], how='left')
compdf = compdf.merge(provdf2, on=['PROVINCE'], how='left')
compdf['MUNSHARE'] = compdf['EMPRESAS']/compdf['MUNICINUM']
compdf['PROVSHARE'] = compdf['EMPRESAS']/compdf['PROVNUM']

## Calculate weighted tariff for each province and municipality
for col in list(compdf.columns)[4:25]: # Might need to check columns
    compdf['wptr'+col] = compdf[col] * compdf['PROVSHARE']
    compdf['wmtr'+col] = compdf[col] * compdf['MUNSHARE']
    
#df.loc[i,'duty'+str(year)]=ctariff(df['Base'][i]

## Sum down to municipality and province tariff averages

# Province level
provtariffdf= compdf
provkeeps = [a for a in provtariffdf.columns if 'wptr' in a]
provkeeps.extend(['PROVINCE'])
provtariffdf = provtariffdf[provkeeps]
provcols = list(provtariffdf.columns)
provcols.remove('PROVINCE')
provtariffdf = provtariffdf.groupby('PROVINCE', as_index=False)[provcols].sum()
provtariffdf.to_csv(provcsv,index=False)

# Municipality level
municitariffdf = compdf
municikeeps = [a for a in municitariffdf.columns if 'wmtr' in a]
municikeeps.extend(['MUNICIPIO'])
municitariffdf = municitariffdf[municikeeps]
municicols = list(municitariffdf.columns)
municicols.remove('MUNICIPIO')
municitariffdf = municitariffdf.groupby('MUNICIPIO', as_index=False)[municicols].sum()
municitariffdf.to_csv(municicsv,index=False)